In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import math

In [ ]:
def logsumexp(x):
    c = x.max()
    return c + np.log(np.sum(np.exp(x - c)))

In [ ]:
class timeVaryingLDS(object):
    def __init__(self,T=None, transition_con = None, emission = None, Q = None, R = None, u0 = None, V0 = None, y=None):
        
        self.y = y # T x N
        self.M = transition_con.shape[1] # dimension of hidden continuous states
        self.T = T # number of observations
        self.N = self.y.shape[1] # dimension of the observations
        
        self.C = emission # the emission probability of the continuous variable, T x N x M
        self.A = transition_con # the continuous variable transition probability matrix, T x M x M 
        
        self.Q = Q # Q is the covariance matrix of noise term added to the hidden continuous state transition, T x M x M
        self.R = R # R is the covariance matrix of noise term added to the emission, T x N x N
        
        self.u0 = u0 # u0 is the initial estimate of the mean of x1, M x 1
        self.V0 = V0 # V0 is the initial estimate of the variance of x1, M x M

        self.P = np.zeros((self.T, self.M, self.M))
        self.P[:,:,:,] = np.eye(self.M)  # P is an intermediate variable during inference, T x M x M
        self.u = np.zeros((self.T, self.M)) # T x M x 1
        self.V = np.zeros((self.T, self.M, self.M)) # T x M x M
        self.K = np.zeros((self.T, self.M, self.N)) # T x M x N
        self.c = np.zeros((self.T)) # T x 1

        # for backward passing
        self.u_hat = np.zeros((self.T, self.M)) # T x M x 1
        self.V_hat = np.zeros((self.T, self.M, self.M)) # T x M x M
        self.J = np.zeros((self.T, self.M, self.M)) # T x M x M


    
    def kalman_filtering(self):
        S_temp = np.matmul(np.matmul(self.C[0], self.V0), self.C[0].T) + self.R[0]
        Q_temp = np.matmul(self.C[0], self.u0)
        I = np.eye(self.M)
        
        self.V[0] = np.matmul((I - np.matmul(np.matmul(np.matmul(self.V0, self.C[0].T), np.linalg.inv(S_temp)), self.C[0])), self.V0)
        self.P[0] = np.matmul(np.matmul(self.A[0], self.V[0]), self.A[0].T) + self.Q[0]
        self.K[0] = np.matmul(np.matmul(self.P[0], self.C[0].T), np.linalg.inv(np.matmul(np.matmul(self.C[0], self.P[0]), self.C[0].T) + self.R[0]))
        self.u[0] = self.u0 + np.matmul(self.K[0], self.y[0] - Q_temp)

        self.c[0] = multivariate_normal.pdf(self.y[0], Q_temp, S_temp)

        for i in range(1,self.T,1):
            I = np.eye(self.M)
            Q_temp = np.matmul(np.matmul(self.C[i], self.A[i]), self.u[i-1])
            
            self.V[i] = np.matmul((I - np.matmul(self.K[i-1], self.C[i])), self.P[i-1])
            self.P[i] = np.matmul(np.matmul(self.A[i], self.V[i]), self.A[i].T) + self.Q[i]
            S_temp = np.matmul(np.matmul(self.C[i], self.P[i]), self.C[i].T) + self.R[i]
            # print('C[i] is',self.C[i],'R[i] is',self.R[i],'A[i] is',self.A[i],'Q[i] is',self.Q[i],'P[i] is',self.P[i],'V[i] is',self.V[i])
            self.K[i] = np.matmul(np.matmul(self.P[i], self.C[i].T), np.linalg.inv(S_temp))

            self.u[i] = np.matmul(self.A[i], self.u[i-1]) + np.matmul(self.K[i-1], self.y[i] - Q_temp)
            # print('i is:',i,'s_temp is:',S_temp)
            self.c[i] = multivariate_normal.pdf(self.y[i], Q_temp, S_temp)

    def kalman_smoothing(self):

        self.u_hat[-1] = self.u[-1]
        self.V_hat[-1] = self.V[-1]

        for i in range(self.T-2,-1,-1):
            # print(i,self.V[i],self.A[i].T,self.P[i])
            self.J[i] = np.matmul(np.matmul(self.V[i], self.A[i].T), np.linalg.inv(self.P[i]))
            self.u_hat[i] = self.u[i] + np.matmul(self.J[i], self.u_hat[i+1] - np.matmul(self.A[i], self.u[i]))
            self.V_hat[i] = self.V[i] + np.matmul(np.matmul(self.J[i], self.V_hat[i+1] - self.P[i]), self.J[i].T)
    
    def kalman_learning(self):
        self.u0 = self.u_hat[0]
        self.V0 = self.V_hat[0] + np.outer(self.u_hat[0], self.u_hat[0].T) - np.outer(self.u_hat[0], self.u_hat[0].T)

        # E[z[n]] : M x 1
        # E[z[n]z[n-1].T] : M x M
        # E[z[n]z[n].T] : M x M

        self.XtXt_1 = np.zeros((self.T,self.M,self.M))
        self.XtXt = np.zeros((self.T,self.M,self.M))
        self.Xt_1Xt = np.zeros((self.T,self.M,self.M))

        self.YtXt = np.zeros((self.T,self.N,self.M))
        self.YtYt = np.zeros((self.T,self.N,self.N))
        self.XtYt = np.zeros((self.T,self.M,self.N))
        
        self.XtXt[0] += self.V_hat[0] + np.outer(self.u_hat[0], self.u_hat[0].T)
        
        for i in range(1,self.T,1):
            
            self.XtXt_1[i] = np.matmul(self.V_hat[i],self.J[i-1].T) + np.outer(self.u_hat[i],self.u_hat[i-1].T) # z[n]z[n-1]
            
            self.XtXt[i] = self.V_hat[i] + np.outer(self.u_hat[i], self.u_hat[i].T) # z[n]z[n]
            self.Xt_1Xt[i] = (np.matmul(self.V_hat[i],self.J[i-1].T) + np.outer(self.u_hat[i],self.u_hat[i-1].T)).T #z[n-1]z[n]

        for i in range(self.T):
            self.YtXt[i] = np.outer(self.y[i], self.u_hat[i].T) # y[n] * E[x[n]].T
            self.YtYt[i] = np.outer(self.y[i], self.y[i].T) # y[n]y[n]
            self.XtYt[i] = np.outer(self.u_hat[i], self.y[i].T) #E[x[n]] * y[n].T 

        sub_1 = np.sum(self.XtXt_1[1:self.T], axis=0)
        sub_2 = np.sum(self.XtXt[0:self.T-1], axis=0)
        sub_3 = np.sum(self.XtXt[1:self.T], axis=0)
        sub_4 = np.sum(self.Xt_1Xt[1:self.T], axis=0)

        sub_5 = np.sum(self.YtXt, axis=0)
        sub_6 = np.sum(self.XtXt, axis=0)
        sub_7 = np.sum(self.YtYt, axis=0)
        sub_8 = np.sum(self.XtYt, axis=0)

        self.A = np.matmul(sub_1, np.linalg.inv(sub_2))

        self.Q = 1/(self.T-1) * (sub_3 - np.matmul(self.A,sub_4) - np.matmul(sub_1,self.A.T) + np.matmul(np.matmul(self.A,sub_2),self.A.T))
        
        self.C = np.matmul(sub_5, np.linalg.inv(sub_6))
        self.R = 1/self.T * (sub_7 - np.matmul(self.C,sub_8) - np.matmul(sub_5,self.C.T) + np.matmul(np.matmul(self.C,sub_6),self.C.T))


In [ ]:
def generate_examples(N):

    np.random.seed(1000)

    # rotate pi / 6 radian in any axis
    A = np.matmul(
        np.matmul(
            np.array([
                [1.0,0.0,0.0],
                [0.0,math.cos(math.pi/6),math.sin(math.pi/6)],
                [0.0,-1.0*math.sin(math.pi/6),math.cos(math.pi/6)]
            ]),
            np.array([
                [math.cos(math.pi/6),0.0,-1.0*math.sin(math.pi/6)],
                [0.0,1.0,0.0],
                [math.sin(math.pi/6),0.0,math.cos(math.pi/6)]
            ])),
        np.array([
            [math.cos(math.pi/6),math.sin(math.pi/6),0.0],
            [-1.0*math.sin(math.pi/6),math.cos(math.pi/6),0.0],
            [0.0,0.0,1.0]
        ])
    )

    Gamma = np.array([
        [1.5, 0.1, 0.0],
        [0.1, 2.0, 0.3],
        [0.0, 0.3, 1.0]
    ])

    Z = np.array([[23.0, 24.0, 25.0]])
    for n in range(N):
        z_prev = Z[len(Z) - 1]
        mean = np.matmul(A, z_prev)
        z_post = np.random.multivariate_normal(
            mean=mean,
            cov=Gamma,
            size=1)
        Z = np.vstack((Z, z_post))

    C = np.array([
        [1.0, 1.0, 0.0],
        [0.0, 1.0, 1.0],
    ])

    Sigma = np.array([
        [1.0,0.2],
        [0.2,2.0],
    ])

    X = np.empty((0,2))
    for z_n in Z:
        mean = np.matmul(C, z_n)
        x_n = np.random.multivariate_normal(
            mean=mean,
            cov=Sigma,
            size=1)
        X = np.vstack((X, x_n))
    return Z, X

In [ ]:
	
n_states = 3 # M
n_obs = 2 # N
n_time = 2000 # T
p_old = -10000
tol = 0.0001
max_iter = 500

# z: T x M
# x : T x N
# A = np.array([[0.9, 0.1],[0.5,0.5]])
# C = np.array([[1, 0],[0.2, 0.8]])
# Gamma = np.array([[0.1, 0.1], [0.1, 0.1]])
# Sigma = np.array([[0.5,0.5],[0.5,0.5]])

A = np.array([[0.75, 0.433, -0.5],[-0.217, 0.875, 0.433],[0.625, -0.217, 0.75]])
Gamma = np.array([[1.5, 0.1, 0.0], [0.1, 2.0, 0.3], [0.0, 0.3, 1.0]])
C = np.array([[1.0,1.0,0.0],[0.0,1.0,1.0]])
Sigma = np.array([[1.0,0.2], [0.2,2.0]])

u0 = np.array([1,2])
V0 = np.array([[0.1,0.3],[0.3,0.1]])
z0 = np.array([[23.0, 24.0, 25.0]])
# A_init = np.array([[0.5, 0.5],[0.5,0.5]])
# C_init = np.array([[0.5, 0.5],[0.5, 0.5]])
# Gamma_init = np.array([[0.5, 0.9], [0.9, 4.5]])
# Sigma_init = np.array([[0.5, 0.9], [0.9, 2.5]])
# u0_init = np.array([1,2])
# V0_init = np.array([[0.2,0.5],[0.5,0.4]])

A_init = np.array([[1.0, 1.1, 1.2],[1.3, 1.4, 1.5],[1.6, 1.7, 1.8]])
A_init = np.tile(A_init,(n_time,1)).reshape(n_time,n_states,n_states)
C_init = np.array([[1.0,1.0,1.0], [1.0, 1.0,1.0]])
C_init = np.tile(C_init,(n_time,1)).reshape(n_time,n_obs,n_states)
Gamma_init = np.array([[1.0, 0.5, 0.5], [0.5,1.0, 0.5],[0.5, 0.5, 1.0]])
Gamma_init = np.tile(Gamma_init,(n_time,1)).reshape(n_time,n_states,n_states)
Sigma_init = np.array([[1.0,0.5], [0.5,1.0]])
Sigma_init = np.tile(Sigma_init,(n_time,1)).reshape(n_time,n_obs,n_obs)
u0_init = np.array([10.0,10.0,10.0])
V0_init = np.array([[1.0, 0.5, 0.5], [0.5,1.0, 0.5],[0.5, 0.5, 1.0]])


# z,x = generate_examples(A = A, C = C, Gamma = Gamma, Sigma = Sigma, z0=z0,M=n_states,N=n_obs,T=n_time)
z,x = generate_examples(n_time)

# kf = KalmanFilter(A = A_init, C = C_init, Gamma = Gamma_init, Sigma = Sigma_init, u0=u0_init, V0=V0_init,x=x, T=n_time)
kf = timeVaryingLDS(T=n_time, transition_con = A_init, emission = C_init, Q = Gamma_init, R = Sigma_init, u0 = u0_init, V0 = V0_init, y=x)
for ite in range(max_iter):
	kf.kalman_filtering()
	kf.kalman_smoothing()        
	kf.kalman_learning()
	kf.A = np.tile(kf.A,(n_time,1)).reshape(n_time,n_states,n_states)
	kf.C = np.tile(kf.C,(n_time,1)).reshape(n_time,n_obs,n_states)
	kf.Q = np.tile(kf.Q,(n_time,1)).reshape(n_time,n_states,n_states)
	kf.R = np.tile(kf.R,(n_time,1)).reshape(n_time,n_obs,n_obs)


	p = np.sum(np.log(kf.c))
	print(f'The current iteration is: {ite}. The likelihood is {p}')
	if p>p_old and p - p_old < tol:
		break
	p_old = p

	